In [1]:
#
# Copyright (c) 2024, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# Distributed Hyperparameter Tuning: Optuna + JoblibSpark


This demo demonstrates distributed hyperparameter tuning for XGBoost using the [JoblibSpark backend](https://github.com/joblib/joblib-spark), building on this [example from Databricks](https://docs.databricks.com/en/machine-learning/automl-hyperparam-tuning/optuna.html).  
We implement best practices to precompute data and maximize computations on the GPU.  



Reference: https://forecastegy.com/posts/xgboost-hyperparameter-tuning-with-optuna/

#### Note:
Before running, please make sure you've followed the relevant [setup instructions](../README.md) for your environment (standalone or databricks).


In [2]:
from typing import List
import os
import requests
import joblib
from joblibspark import register_spark
import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
from pyspark.sql import SparkSession
from pyspark import TaskContext, SparkConf

### Download the dataset

We'll use the [red wine quality dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) to regress wine quality based on features such as acidity, sugar content, etc.  

**Note**: This example uses a small dataset for demonstration purposes. The performance advantages of distributed training are best realized with large datasets and computational workloads.

In [3]:
cwd = os.getcwd()
os.mkdir(os.path.join(cwd, "data")) if not os.path.exists(os.path.join(cwd, "data")) else None
filepath = os.path.join(cwd, "data", "winequality-red.csv")

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

response = requests.get(url)
if response.status_code == 200:
    with open(filepath, "wb") as f:
        f.write(response.content)
    print(f"File downloaded and saved to {filepath}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved to /home/rishic/Code/myforks/spark-rapids-examples/examples/ML+DL-Examples/Optuna-Spark/optuna-examples/data/winequality-red.csv


## Part 1. Running Optuna locally

In [5]:
import cudf
from cuml.metrics.regression import mean_squared_error
from cuml.model_selection import train_test_split

### Prepare data

In [6]:
data = cudf.read_csv(filepath, delimiter=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Prepare the train/validation sets. Precompute the Quantile DMatrix, which is used by histogram-based tree methods to save memory.

In [7]:
X = data.iloc[:, :-1].values
y = data["quality"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
Xy_train_qdm = xgb.QuantileDMatrix(X_train, y_train)  # Precompute Quantile DMatrix to avoid repeated quantization every trial.

### Objective function

We define the objective and a hyperparameter search space to optimize via the `trial.suggest_` methods.  

In each trial, new hyperparameters will be suggested based on previous results. See [optuna.trial.Trial](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html) API for a full list of functions.

In [9]:
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "tree_method": "gpu_hist",
        "device": "cuda",
    }

    booster = xgb.train(params=params, dtrain=Xy_train_qdm, num_boost_round=trial.suggest_int("num_boost_round", 100, 500))
    predictions = booster.inplace_predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False).get()
    
    return rmse   

Create the study and optimize. By default, the study results will be stored in memory.

In [10]:
study = optuna.create_study(study_name="optuna-xgboost-local", sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=10)

[I 2024-12-11 23:42:09,341] A new study created in memory with name: optuna-xgboost-local


[I 2024-12-11 23:42:09,715] Trial 0 finished with value: 0.6377619522504244 and parameters: {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'subsample': 0.7453942447208348, 'colsample_bytree': 0.6187255599871848, 'min_child_weight': 4, 'num_boost_round': 162}. Best is trial 0 with value: 0.6377619522504244.
[I 2024-12-11 23:42:10,666] Trial 1 finished with value: 0.6703788974319568 and parameters: {'learning_rate': 0.0013066739238053278, 'max_depth': 9, 'subsample': 0.6210592611560484, 'colsample_bytree': 0.7226689489062432, 'min_child_weight': 1, 'num_boost_round': 488}. Best is trial 0 with value: 0.6377619522504244.
[I 2024-12-11 23:42:10,806] Trial 2 finished with value: 0.6181751362616256 and parameters: {'learning_rate': 0.04622589001020832, 'max_depth': 3, 'subsample': 0.2227337188467456, 'colsample_bytree': 0.22423428436076215, 'min_child_weight': 7, 'num_boost_round': 310}. Best is trial 2 with value: 0.6181751362616256.
[I 2024-12-11 23:42:10,922] Trial 3 finished wi

In [11]:
trial = study.best_trial
print("Best RMSE: ", trial.value)
print("Best hyperparameters: ", trial.params)

Best RMSE:  0.6060010014477214
Best hyperparameters:  {'learning_rate': 0.0123999678368461, 'max_depth': 2, 'subsample': 0.9711053963763306, 'colsample_bytree': 0.7863761821930588, 'min_child_weight': 19, 'num_boost_round': 458}


## Part 2. Distributed Optuna on Spark 

### PySpark

For standalone users, we need to create the Spark session. For Databricks users, the Spark session will be preconfigured.

In [12]:
def initialize_spark():
    import socket
    hostname = socket.gethostname()
    conda_env = os.environ.get("CONDA_PREFIX")

    conf = SparkConf()
    conf.setMaster(f"spark://{hostname}:7077")  # Assuming master is on host and default port. 
    conf.set("spark.task.maxFailures", "1")
    conf.set("spark.task.resource.gpu.amount", "1")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
    conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
    
    spark = SparkSession.builder.appName("optuna-joblibspark-xgboost").config(conf=conf).getOrCreate()
    return spark

if 'spark' not in globals():
    spark = initialize_spark()

24/12/11 23:42:12 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
24/12/11 23:42:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 23:42:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Optuna Task

This implementation demonstrates **Worker I/O**. 

This means that each worker will read the full dataset from the filepath rather than passing the data in a dataframe.  
In practice, this requires the dataset to be written to a distributed file system accessible to all workers prior to tuning. 

For the alternative implementation using **Spark I/O**, see the [Spark Dataframe notebook](optuna-dataframe.ipynb).

In the task, each worker will:
1. Read the dataset from the filepath
2. Load the study from the MySQL storage backend
3. Optimize over the objective for the assigned number of trials, sending results back to the database after each iteration

Here we use Optuna's [Define-and-Run](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/009_ask_and_tell.html#define-and-run) API, which allows us to predefine the hyperparameter space and pass it to the task.

In [13]:
def task(num_trials: int, xgb_params: dict, optuna_params: dict, driver_ip: str, study_name: str, seed: int, filepath: str):
    import cudf
    from cuml.metrics.regression import mean_squared_error
    from cuml.model_selection import train_test_split

    tc = TaskContext.get()
    assert "gpu" in tc.resources(), "GPU resource not found."

    if filepath.startswith("/dbfs/"):
        # Check to ensure GPU direct storage is disabled for cuDF on databricks.
        libcudf_policy = os.environ.get('LIBCUDF_CUFILE_POLICY')
        if libcudf_policy != 'OFF':
            raise RuntimeError("Set LIBCUDF_CUFILE_POLICY=OFF to read from DBFS with cuDF.")
    
    data = cudf.read_csv(filepath, delimiter=";")
    X = data.iloc[:, :-1].values
    y = data["quality"].values
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

    tuning_max_bin = "max_bin" in optuna_params
    if not tuning_max_bin:
        max_bin = xgb_params.get("max_bin", 256)
        # Precompute Quantile DMatrix to avoid repeated quantization every trial.
        Xy_train_qdm = xgb.QuantileDMatrix(X_train, y_train, max_bin=max_bin)

    study = optuna.load_study(
        study_name=study_name,
        storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna",
        sampler=TPESampler(seed=seed),
    )

    print(f"Running {num_trials} trials on partition {tc.partitionId()}.")

    ### Objective ###
    for _ in range(num_trials):
        trial = study.ask(optuna_params)
        xgb_params.update(trial.params)

        if tuning_max_bin:
            # If tuning the max_bin param, we must recompute the QDM every trial.
            if "n_estimators" not in xgb_params:
                xgb_params["n_estimators"] = 100  # Default value if not tuning.

            model = xgb.XGBRegressor(**xgb_params)
            model.fit(X_train, y_train)
            booster = model.get_booster()
        else:
            # Train the model with xgb.train() API using the precomputed QDM.
            num_boost_round = xgb_params.get("n_estimators", 100)
            booster = xgb.train(params=xgb_params, dtrain=Xy_train_qdm, num_boost_round=num_boost_round)
            
        # Perform in-place predictions on GPU using the booster.
        predictions = booster.inplace_predict(X_val)
        rmse = mean_squared_error(y_val, predictions, squared=False).get()
        
        study.tell(trial, rmse)

    return study.best_params, study.best_value

In [14]:
# This will register the Spark Session with the Joblib Spark backend.
register_spark()

## Setup and run the Optuna study

Get the driver IP for the MySQL database.  
- For standalone users, make sure you've followed the [database setup instructions](../README.md#setup-database-for-optuna). The database should be on 'localhost'. 
- For databricks users, the database should already be setup on the driver node by the init script.

In [15]:
# check if we're running on databricks
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)

In [16]:
if on_databricks:
    driver_ip = spark.conf.get("spark.driver.host")
else:
    driver_ip = "localhost"

print(f"MySQL database is hosted on {driver_ip}")

MySQL database is hosted on localhost


Create a new study, referencing the MySQL database as the storage backend.

In [17]:
study_name = "optuna-xgboost-joblibspark"
seed = 42

try:
    # Delete the study if it already exists
    optuna.delete_study(
        study_name=study_name, 
        storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna"
    )
except:
    pass

optuna.create_study(
    study_name=study_name,
    storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna",
    sampler=TPESampler(seed=seed)
)

[I 2024-12-11 23:42:13,928] A new study created in RDB with name: optuna-xgboost-joblibspark


Define the number of tasks, number of trials, and trials per task. 

**NOTE**: for standalone users running on a single worker, the 4 tasks will all be assigned to the same worker and executed sequentially in this demonstration. This can easily be scaled up to run concurrently by adding more workers.

In [18]:
def partition_trials(total_trials: int, total_tasks: int) -> List[int]:
    base_size = total_trials // total_tasks
    extra = total_trials % total_tasks
    partitions = [base_size] * total_tasks
    for i in range(extra):
        partitions[i] += 1
    
    return partitions

In [19]:
num_tasks = 4
num_trials = 100
trials_per_task = partition_trials(num_trials, num_tasks)
print(f"Trials per task: {trials_per_task}")

Trials per task: [25, 25, 25, 25]


#### Define params
Define the XGBoost model params and the hyperparams for Optuna to tune. 

In [ ]:
# Keep these params consistent:
xgb_params = {
    "objective": "reg:squarederror",
    "verbosity": 0,
    "tree_method": "gpu_hist",
    "device": f"cuda",
    "seed": seed,
}

In [ ]:
# Tune these params:
optuna_params = {
    "n_estimators": optuna.distributions.IntDistribution(100, 500),
    "learning_rate": optuna.distributions.FloatDistribution(1e-3, 0.1, log=True),
    "max_depth": optuna.distributions.IntDistribution(1, 10),
    "subsample": optuna.distributions.FloatDistribution(0.05, 1.0),
    "colsample_bytree": optuna.distributions.FloatDistribution(0.05, 1.0),
    "min_child_weight": optuna.distributions.IntDistribution(1, 20),
}

**For Databricks**: we must download the dataset to DBFS so that all workers can access it.

In [22]:
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/optuna-data")
    filepath = "/dbfs/FileStore/optuna-data/winequality-red.csv"
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

    response = requests.get(url)
    if response.status_code == 200:
        with open(filepath, "wb") as f:
            f.write(response.content)
        print(f"File downloaded and saved to {filepath}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

### Run the study

Run parallel threads to execute the Optuna task and collect the reuslts (it might take a few minutes).

In [23]:
with joblib.parallel_backend("spark", n_jobs=num_tasks):
    results = joblib.Parallel()(
        joblib.delayed(task)(num_trials,
                             xgb_params,
                             optuna_params,
                             driver_ip,
                             study_name,
                             seed,
                             filepath) for num_trials in trials_per_task
    )

/home/rishic/anaconda3/envs/optuna-spark/lib/python3.10/site-packages/joblibspark/backend.py:115: UserWarning: Spark version does not support stage-level scheduling.
  warnings.warn("Spark version does not support stage-level scheduling.")
/home/rishic/anaconda3/envs/optuna-spark/lib/python3.10/site-packages/joblibspark/backend.py:154: UserWarning: User-specified n_jobs (4) is greater than the max number of concurrent tasks (1) this cluster can run now.If dynamic allocation is enabled for the cluster, you might see more executors allocated.
  warnings.warn(f"User-specified n_jobs ({n_jobs}) is greater than the max number of "


In [24]:
best_params = min(results, key=lambda x: x[1])[0]
best_value = min(results, key=lambda x: x[1])[1]

print(f"Best parameters: {best_params}")
print(f"Best value: {best_value}")

Best parameters: {'n_estimators': 463, 'learning_rate': 0.05206124631137337, 'max_depth': 9, 'subsample': 0.7434942725744815, 'colsample_bytree': 0.877391644494205, 'min_child_weight': 4}
Best value: 0.5324732150787205
